In [1]:
import os, pickle
import numpy as np
import pandas as pd
import sys, os, torch
from pathlib import Path
from typing import List, Dict, Tuple, Union, Optional, Any, BinaryIO
from sklearn.metrics import (precision_score, recall_score,
                             average_precision_score, f1_score,
                             precision_recall_curve)

sys.path.append(os.path.abspath("../"))
from utils.general_utils import load_pickle, readVidEvents, get_file_name
from misc.all_label_generator import triangle_smoother, get_smoothed_labels, vidSeg2labels


/home/rodo/miniconda3/envs/rodo_main/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Sec 1

**Generate AP score for each episode in `24`...**

In [47]:
def generate_APs(series:str, season:str, episode:str, seg_start, seg_end, window_sizes=list(range(2, 15)), modality='vid'):
    """
    Generate different APs for different window sizes.
    """
    next_ep = "E%02d" % (int(episode[-2:]) + 1)
    if modality == 'vid':
        gl = load_pickle(f"../data/{series}/{season}/{season}{next_ep}/scores/"+\
                         "vid_scores/recapVepisode/GT.pkl")[episode]
    else:
        gl = load_pickle(f"../data/{series}/{season}/{season}{next_ep}/scores/"+\
                         "dia_scores/recapVepisode/GT.pkl")[episode]

    # generate manual labels
    if modality == 'vid':
        ml = vidSeg2labels(seg_start, seg_end, series, season, episode)
    elif modality == 'dia':
        dia_file = get_file_name(fr"../data/{series}/{season}/{season}{next_ep}/encodings/dia_encodings/newSrtObj_*.pkl")
        start = int(dia_file.stem.split("_")[-1]) + 1
        ml = load_pickle(f"../data/{series}/{season}/{season}{next_ep}/scores/"+\
                         "dia_scores/recapVepisode/SLV.pkl")[episode][start:]
        ml = (ml>0.5).astype(np.int32)
        gl = gl[start:]
        assert len(gl) == len(ml), f"Length of Ground-Truth doesn't match with Manual Labels"

    APs = {}
    for k in window_sizes:
        sl = get_smoothed_labels(gl, half_base_len=k)
        AP = average_precision_score(ml, sl)
        APs[k] = AP
    APs['best'] = max(APs, key = lambda x: APs[x])
    return APs

# WE GOT HALF_BASE_LEN = 10 AS BEST

# so we've Ground-Truth labels (gl), Smoothed labels (sl), and Manual labels (ml)
# WORK TODO: compare the three labels
# Steps:
# 1. Compare gl and ml; Compute precision, recall, f1 (Just to get idea - how worse is it?)
# 2. Compare sl and ml; Compute AP. Based on best threshold from P-R curve, compute precision, recall, f1
# 3. Vary `half_base_len` and see how it affects the AP, f1.
#    Choose that value which gives best results.
# Note: By default, we assume shots from same segment can't be more than 10 shot-distances apart. Hence threshold of 11 has been used while generating ground-truth labels in shotFinder() function.

In [46]:
fandom_time_new_segs = {'S02E21': {'beg': [243.389, 336.44, 492.263, 692.463, 1064.042, 1854.04, 2338.566, 2468.195, 2568.629],
            'end': [272.752, 353.374, 535.264, 732.002, 1118.096, 1917.52, 2453.639, 2509.486, 2605.749], 'fps': 23.976023976023978},
 'S02E23': {'beg': [596.242, 745.057, 1296.817, 1464.109, 1683.703, 1800.403, 1871.224, 2237.673, 2457.685, 2515.075],
            'end': [646.458, 805.826, 1353.206, 1522.542, 1720.907, 1830.85, 1945.631, 2365.175, 2491.551, 2544.104], 'fps': 23.976023976023978},
 'S03E20': {'beg': [222.034, 814.626, 901.38, 957.561, 2014.951, 2381.775, 2508.819],
            'end': [265.119, 867.679, 924.82, 1052.239, 2075.761, 2431.617, 2585.479], 'fps': 23.976023976023978},
 'S03E22': {'beg': [113.301, 418.814, 644.498, 1622.892, 2147.291, 2226.162, 2236.839, 2286.931, 2371.974],
            'end': [140.911, 502.856, 721.325, 1679.407, 2211.271, 2231.125, 2279.339, 2329.973, 2409.928], 'fps': 23.976023976023978},
 'S04E20': {'beg': [251.564, 389.994, 716.403, 789.518, 1213.608, 1492.721, 1585.814, 1769.497, 1996.182, 2230.416, 2300.611, 2473.867, 2533.761],
            'end': [296.859, 416.145, 771.75, 879.274, 1266.327, 1560.038, 1646.04, 1807.368, 2069.129, 2256.233, 2397.082, 2502.145, 2560.954], 'fps': 23.976023976023978},
 'S04E21': {'beg': [139.869, 691.837, 1018.372, 1129.9, 2101.912, 2344.989, 2379.481],
            'end': [160.056, 736.84, 1054.783, 1196.716, 2179.615, 2377.187, 2595.363], 'fps': 23.976023976023978},
 'S05E21': {'beg': [623.811, 871.809, 1519.998, 1562.957, 1808.578, 1866.219, 2008.903, 2076.012, 2413.641, 2545.856],
            'end': [722.242, 1110.797, 1527.839, 1603.831, 1837.815, 1897.5, 2053.113, 2088.315, 2498.934, 2559.327], 'fps': 23.976023976023978},
 'S05E22': {'beg': [434.413, 677.239, 968.655, 1113.425, 1193.171, 1531.885, 2087.398, 2266.368, 2363.549, 2418.52, 2459.979, 2516.577],
            'end': [481.585, 765.035, 999.728, 1127.689, 1257.402, 1607.627, 2098.325, 2312.956, 2400.21, 2448.05, 2471.365, 2569.046], 'fps': 23.976023976023978},
 'S06E20': {'beg': [557.495, 716.195, 852.956, 1153.382, 1386.489, 1558.995, 1879.857, 2016.786, 2178.364, 2505.149],
            'end': [675.654, 767.537, 950.262, 1231.793, 1484.837, 1595.114, 1900.586, 2038.724, 2316.71, 2567.21], 'fps': 23.976023976023978},
 'S06E21': {'beg': [741.387, 1058.912, 1154.299, 1540.185, 1789.851, 1857.46, 1864.759, 1945.756, 2015.576, 2104.624, 2338.023, 2514.45],
            'end': [786.056, 1084.02, 1179.157, 1589.108, 1831.392, 1864.717, 1891.786, 1975.119, 2056.158, 2191.585, 2417.853, 2552.779], 'fps': 23.976023976023978},
 'S06E22': {'beg': [157.553, 553.532, 698.844, 818.422, 1543.855, 2072.883, 2309.537, 2394.204, 2525.544],
            'end': [179.617, 667.02, 717.863, 911.181, 1602.83, 2216.276, 2356.208, 2478.956, 2556.366], 'fps': 23.976023976023978},
 'S06E23': {'beg': [319.882, 564.251, 1470.407, 1736.464, 1959.979, 2290.142, 2444.797],
            'end': [390.911, 637.783, 1521.374, 1779.84, 2033.677, 2342.152, 2475.702], 'fps': 23.976023976023978},
 'S07E20': {'beg': [486.09, 644.415, 743.889, 895.082, 1254.608, 2142.662, 2300.861, 2560.788],
            'end': [585.355, 712.065, 892.079, 1053.615, 1430.992, 2272.332, 2397.666, 2587.105], 'fps': 23.976023976023978},
 'S07E22': {'beg': [805.826, 1153.548, 1322.133, 1591.778, 1737.924, 1811.539, 2430.24],
            'end': [987.507, 1222.909, 1374.727, 1689.709, 1753.981, 1849.66, 2485.003], 'fps': 23.976023976023978},
 'S07E23': {'beg': [176.739, 240.928, 274.045, 428.366, 660.055, 731.961, 852.373, 906.635, 2089.859, 2277.797, 2358.836, 2478.622],
            'end': [226.372, 258.279, 311.207, 546.9, 701.597, 826.43, 863.8, 918.438, 2198.926, 2324.134, 2442.419, 2494.429], 'fps': 23.976023976023978},
 'S08E21': {'beg': [589.276, 776.046, 852.331, 1137.908, 1490.927, 2027.296, 2129.19, 2312.873, 2536.013],
            'end': [635.03, 812.958, 903.507, 1228.289, 1670.857, 2071.966, 2145.122, 2513.782, 2591.985], 'fps': 23.976023976023978},
 'S08E22': {'beg': [285.139, 459.647, 838.15, 1200.429, 1423.443, 1815.501, 2111.297, 2303.823, 2411.347],
            'end': [380.109, 515.202, 885.28, 1260.572, 1513.950, 1873.100, 2226.995, 2371.557, 2528.964], 'fps': 23.976023976023978}}

In [ ]:
# Estimate shot number from the time-segments given above using vid_events file...
shots_segs = {}
for key, val in fandom_time_new_segs.items():
    season = key[:3]
    episode  = key[-3:]
    vid_events = readVidEvents(f"../data/24/{season}/{key}/videvents/{key}.videvents")
    shots_segs[key] = {'beg':[], 'end':[]}
    for start, end in zip(val['beg'], val['end']):
        k = 0; tmp_lst = []
        while vid_events[k][1] < start:
            k += 1
        while start <= vid_events[k][1] < end:
            tmp_lst.append(k+1)# as in videvents shot index 1 is there not 0th.
            k += 1
        shots_segs[key]['beg'].append(tmp_lst[0])
        shots_segs[key]['end'].append(tmp_lst[-1])

print(shots_segs)

In [ ]:
""" module_mapping = {'vid_utils':'utils.vid_utils'}
update_module_references("../data/24/S02/S02E02/encodings/vid_encodings/episode_OBJ.pkl",
                         "../data/24/S02/S02E02/encodings/vid_encodings/episode_OBJ.pkl",
                         module_mapping)
 obj = load_pickle("../data/24/S02/S02E02/scores/vid_scores/recapVepisode/GT.pkl")"""

##### Evaluate which window size is best for smoothing...

In [48]:
APs = {}
for ep, time in shots_segs.items():
    APs[ep] = generate_APs(season=ep[:3], series=24, episode=ep[-3:], seg_start=time['beg'], seg_end=time['end'], modality='vid')

APs

{'S02E21': {2: 0.3773673135375263,
  3: 0.4034595458610251,
  4: 0.41280307254260723,
  5: 0.4133169278122555,
  6: 0.41711545543903444,
  7: 0.42440101924168255,
  8: 0.4197049506255487,
  9: 0.40769946491947956,
  10: 0.4035321667142382,
  11: 0.397123106031443,
  12: 0.38935395706167214,
  13: 0.3830397472039517,
  14: 0.37873059103957085,
  'best': 7},
 'S02E23': {2: 0.4848116377287579,
  3: 0.5371016656217018,
  4: 0.561485780902815,
  5: 0.5807526742659725,
  6: 0.5838543804804524,
  7: 0.5985530168173724,
  8: 0.5943656166708395,
  9: 0.5922528076609288,
  10: 0.5846227845288217,
  11: 0.5798132139891182,
  12: 0.5714691170261306,
  13: 0.5597905017326373,
  14: 0.5413220991057369,
  'best': 7},
 'S03E20': {2: 0.7293772032902468,
  3: 0.7412564942268297,
  4: 0.7755805167379797,
  5: 0.7473394954237295,
  6: 0.7226013647520073,
  7: 0.6953010699104596,
  8: 0.6736390267574512,
  9: 0.639310002827619,
  10: 0.5928012824911488,
  11: 0.588681389162159,
  12: 0.5732019370033014,
  

In [52]:
AP_lst = [APs[k][APs[k]['best']] for k in APs]
print(f"Mean AP = {np.mean(AP_lst)}; Stddev = {np.std(AP_lst)}")

Mean AP = 0.6449494502131495; Stddev = 0.0823425627465252


In [53]:
optimal_window_size_vid = np.mean([APs[k]['best'] for k in APs])
optimal_window_size_vid

6.0588235294117645

In [43]:
APs_dia = {}
for ep, time in shots_segs.items():
    APs_dia[ep] = generate_APs(season=ep[:3], series=24, episode=ep[-3:], seg_start=time['beg'], seg_end=time['end'], modality='dia')
optimal_window_size_dia = np.mean([APs_dia[k]['best'] for k in APs_dia])
optimal_window_size_dia

9.294117647058824

In [54]:
APs_dia_lst = [APs_dia[k][APs_dia[k]['best']] for k in APs_dia]
print(f"Mean AP = {np.mean(APs_dia_lst)}; Stddev = {np.std(APs_dia_lst)}")

Mean AP = 0.7268071542307943; Stddev = 0.0772461076731156


In [11]:
# # for 23 from 22
# seg_start_22 = [47, 144, 175, 217, 496, 679, 743, 782, 802]
# seg_end_22 = [57, 161, 178, 227, 505, 730, 757, 792, 810]

# # for 21 from 20
# seg_start_20 = [79, 247, 266, 282, 608, 708, 749]
# seg_end_20 = [89, 259, 273, 305, 625, 730, 776]

### Sec 2

In [2]:
def count_positives(series, seasons, total_eps, modality="vid", verbose=0):
    global_dict = {}
    for season, total_ep in zip(seasons, total_eps):
        path = f"../data/{series}/{season}/"
        all_eps = ["E%02d"%k for k in range(2, total_ep+1)]
        ratio_dict = {}
        for ep in all_eps:
            prev_ep = "E%02d"%(int(ep[-2:])-1)
            try:
                gt_vid = load_pickle(path + season + ep + "/scores/vid_scores/recapVepisode/SL.pkl")[prev_ep]
            except FileNotFoundError:
                continue
            try:
                file_name = get_file_name(path + season + ep + "/encodings/dia_encodings/newSrtObj_*.pkl")
            except FileNotFoundError:
                continue
            boundary = int(file_name.stem.split("_")[-1])+1
            try:
                gt_dia = load_pickle(path + season +  ep + "/scores/dia_scores/recapVepisode/SLV.pkl")[prev_ep][boundary:]
            except FileNotFoundError:
                continue
            if verbose and modality == "vid":
                print(f"Total shots in {prev_ep} = ",len(gt_vid))
                print(f"No. of shots from {prev_ep} as RECAP in {ep} = ",sum(gt_vid))
                print(f"No. of Negative shots in {prev_ep} = {len(gt_vid) - sum(gt_vid)}")
                print("-"*100)
            elif verbose and modality == "dia":
                print(f"Total dialog in {prev_ep} = ",len(gt_dia))
                print(f"No. of dialog from {prev_ep} as RECAP in {ep} = ",sum(gt_dia))
                print(f"No. of Negative dialog in {prev_ep} = {len(gt_dia) - sum(gt_dia)}")
                print("-"*100)
            elif verbose and modality == "both":
                print(f"Total shots + dia in {prev_ep}",len(gt_vid)+len(gt_dia))
                print(f"No. of shots + dia from {prev_ep} as RECAP in {ep}",sum(gt_vid)+sum(gt_dia))
                print("-"*100)
            # ratio of (negative / positive) labels = will be the importance for positive examples
            if modality == "vid":
                ratio_dict[prev_ep] = (len(gt_vid)-sum(gt_vid))/sum(gt_vid)
            elif modality == "dia":
                ratio_dict[prev_ep] = (len(gt_dia)-sum(gt_dia))/sum(gt_dia)
            else:
                ratio_dict[prev_ep] = (len(gt_vid)+len(gt_dia)-sum(gt_vid)-sum(gt_dia))/(sum(gt_vid)+sum(gt_dia))
        print(f"Averaged shots from previous episode as RECAP in next episode",np.mean(list(ratio_dict.values())))
        global_dict[season] = ratio_dict
    global_lst = []
    for v in global_dict.values():
        global_lst += list(v.values())
    print(f"Overall weightage to positive labels in {modality.upper()} modality = {np.mean(global_lst)} +- {np.std(global_lst)}")
    print("-"*100)
    return global_dict

**24-Video/Dialog/Both Shots `+ve` v/s `-ve` count.**

In [3]:
# series_24 = "24"
# seasons_24 = ["S02", "S03", "S04", "S05", "S06", "S07", "S08", "S09"]
# total_eps_24 = [24, 24, 24, 24, 24, 24, 24, 12]

series_24 = "prison-break"
seasons_24 = ["S02", "S03"]
total_eps_24 = [22, 13]

In [7]:
# _ = count_positives(series_24, seasons_24, total_eps_24, verbose=0)
# _ = count_positives(series_24, seasons_24, total_eps_24, modality='dia', verbose=0)
_ = count_positives(series_24, seasons_24, total_eps_24, modality='both', verbose=0)

Averaged shots from previous episode as RECAP in next episode 3.786063826039521
Averaged shots from previous episode as RECAP in next episode 5.1637255563730555
Overall weightage to positive labels in BOTH modality = 4.287031727978989 +- 1.5305004264957
----------------------------------------------------------------------------------------------------


**PB-Video/Dialog/Both Shots `+ve` v/s `-ve` count.**

In [37]:
series_PB = "prison-break"
seasons_PB = ["S02", "S03",]
total_eps_PB = [22, 13]

In [38]:
_ = count_positives(series_PB, seasons_PB, total_eps_PB)
_ = count_positives(series_PB, seasons_PB, total_eps_PB, modality='dia')
_ = count_positives(series_PB, seasons_PB, total_eps_PB, modality='both')

Averaged shots from previous episode as RECAP in next episode 3.446486026239614
Averaged shots from previous episode as RECAP in next episode 4.999294249321901
Overall weightage to positive labels in VID modality = 3.9974824924946204
----------------------------------------------------------------------------------------------------
Averaged shots from previous episode as RECAP in next episode 25.956381713330405
Averaged shots from previous episode as RECAP in next episode 34.11261990302173
Overall weightage to positive labels in DIA modality = 28.85053074838217
----------------------------------------------------------------------------------------------------
Averaged shots from previous episode as RECAP in next episode 5.3227365384911085
Averaged shots from previous episode as RECAP in next episode 6.891855972340815
Overall weightage to positive labels in BOTH modality = 5.879520853728101
-----------------------------------------------------------------------------------------------